In [20]:
# IMPORTACOES DE MODULOS E BIBLIOTECAS

from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.error import HTTPError
import csv

In [21]:
# FUNCOES

def getPage(url):
  try:
    resposta = urlopen(url)
  except HTTPError as e:
    return None
  return resposta

In [ ]:
links= []
counts=[]
counttemp = []
datas= []
dicionario = {}
link_base =" http://www.nuforc.org/webreports/ndxevent.html "
link_suporte = "http://www.nuforc.org/webreports/"
resposta_http = getPage(link_base)
objeto_soup = BeautifulSoup(resposta_http.read(), features="html.parser")

#Pegando as DATAS, LINKS
for link in objeto_soup.find_all('a')[1:]:
  if link.get_text()=='12/1996':
    break
  elif int(link.get_text()[3:])>2017:
    continue
  datas.append((link.get_text()))
  links.append((link.get('href')))

datas = datas[4:-8]
links = links[4:-8]

#Pegando os COUNTS
for link in objeto_soup.find_all('td'):
  counttemp.append((link.get_text()))
counttemp = {counttemp[i]: counttemp[i+1] for i in range(0, len(counttemp), 2)}
    
for i in counttemp:
  if i=='12/1996':
    break
  elif(int(i[3:])>2017):
    continue
  counts.append(counttemp[i])
counts = counts[4:-8]

# Montando o dicionário inicial
for i in range(0, len(datas)):
  dicionario[datas[i]] = {  
    "count":counts[i],
    "link":links[i]
  }

print(dicionario)

In [ ]:
# dicionário que guarda as informações de todos os registros coletados no site
ovnis = {}
chave = 0

# percorre todo o dicionário
for i in dicionario:
  # lista que armazena as linhas das tabelas de cada página/link acessado
  linhas = []
  # acessa a página correspondente ao link em questão
  pagina = link_suporte+dicionario[i]['link']
  resposta = getPage(pagina)
  soup = BeautifulSoup(resposta.read(), features="html.parser")
  linhas_tabela = soup.find_all("tr")
  
  for l in linhas_tabela:
    linhas.append(l.get_text().strip().split('\n'))

  # Armazenando as informações no dicionário principal (pré-csv)
  for linha in linhas[1:]:
    ovnis[str(chave)] = {
        'data_hora': linha[0],
        'cidade': linha[1],
        'estado': linha[2],
        'formato': linha[3],
        'duracao': linha[4],
        'resumo': linha[5],
        'data_postagem': linha[6]
    }
    chave += 1

  print("Ovnis: ", len(ovnis))

  # Limitador para teste
  '''if(len(ovnis)>=1367):
    break'''

In [25]:
# CELULA RESPONSAVEL POR CONVERTER O DICT PARA CSV

csv_columns = ['ID', 'data_hora' , 'cidade', 'estado', 'formato', 'duracao', 'resumo', 'data_postagem']
csvfile = 'OVNIS.csv'
dict_data = [
{'ID': i,
 'data_hora': ovnis[i]['data_hora'],
 'cidade': ovnis[i]['cidade'],
 'estado': ovnis[i]['estado'],
 'formato': ovnis[i]['formato'],
 'duracao': ovnis[i]['duracao'],
 'resumo': ovnis[i]['resumo'],
 'data_postagem': ovnis[i]['data_postagem']} for i in ovnis
]

try:
  with open(csvfile, 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
    writer.writeheader()
    for data in dict_data:
      writer.writerow(data)
except IOError:
  print("I/O error")
